### IBM Data Science Capstone Project (Coursera )
Opening new catering service outlets in Pune, India

The project is for the IBM Data Science Specialization on Coursera. To understand the full purpose of the code please refer to the final repot linked below:

https://github.com/VatsalUnadkat/IBM_Data_Science_Capstone_Project

In [132]:
import sys
import folium
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import json
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [133]:
# Foursquare credentials 
CLIENT_ID = "XMCQHNI0OZUHUWCJ2342ZW5ROMBRPTIFSFHF0ANND3ZAT5AB"
CLIENT_SECRET = "04YCAO4TD2Z11E4XDWZ3Z0Y4UXY1FIEPAFQBID2DMLX1CX0W"
VERSION = "20180605"

### Extracting the data


In [134]:
# Function to scrape data from website and convert it into dataframe
geolocator = Nominatim(user_agent="coursera agent")
def create_areas_data_frame(): 
    with open('area_index.html') as f:
        soup = BeautifulSoup(f, parser='lxml')

    city_areas = soup.find_all('div', attrs={'class': 'city-listing'})
    areas_dict = {'Area': [], 'Latitude': [], 'Longitude': []}
    for areas in city_areas:
        area_cat = areas.find('p', attrs={'class': 'key-values'})
        for area_parsed in area_cat.find_all('a'):
            area_short = area_parsed.text.split(',')[0]
            area = '{}, Pune'.format(area_short)
            try:
                location = geolocator.geocode(area)
                latitude = location.latitude
                longitude = location.longitude
            except Exception as e:
                pass
            else:
                areas_dict['Area'].append(area_short)
                areas_dict['Latitude'].append(latitude)
                areas_dict['Longitude'].append(longitude)

    df_areas = pd.DataFrame(areas_dict)
    # df_areas.to_csv('pune_areas.csv')
    return df_areas

In [135]:
# Load scraped data from csv
df_pune = pd.read_csv('pune_areas.csv')
df_pune.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df_pune = df_pune[(df_pune['Latitude'] < 18.6) & (df_pune['Latitude'] > 18.35)
                  & (df_pune['Longitude'] < 74.0) & (df_pune['Longitude'] > 73.6)]
print(df_pune.shape)
df_pune.head(10)

(69, 3)


Area   Latitude  Longitude
0              Aundh  18.561883  73.810196
3        Anand Nagar  18.478490  73.821326
5           Balewadi  18.582027  73.768983
6              Baner  18.564243  73.776857
7   Bund Garden Road  18.533404  73.878625
10      Budhwar Peth  18.517560  73.858037
11             Baner  18.564243  73.776857
12      Central Pune  18.520306  73.854319
17  Dhole Patil Road  18.534908  73.876258
18   Deccan Gymkhana  18.515893  73.841160

#### There are a total 69 areas in Pune.

In [136]:
location = geolocator.geocode('Pune')
pune_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)
for lat, lon, area in zip(df_pune['Latitude'], df_pune['Longitude'], df_pune['Area']):
    label = folium.Popup(area)
    folium.CircleMarker(
        [lat, lon],
        color='blue',
        popup=label,
        radius=8,
        fill_color='yellow',
        fill=True,
        fill_opacity=0.8
    ).add_to(pune_map)
pune_map

In [137]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=restaurant'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4>Get list of restaurants under each area</h4>

In [138]:
pune_venues = getNearbyVenues(names=df_pune['Area'],
                                   latitudes=df_pune['Latitude'],
                                   longitudes=df_pune['Longitude']
                                  )

Aundh
Anand Nagar
Balewadi
Baner
Bund Garden Road
Budhwar Peth
Baner
Central Pune
Dhole Patil Road
Deccan Gymkhana
Dapodi
Fergusson College Road
Fursungi
Fatima Nagar
Ghorpadi
Gultekdi
Hadapsar
Hinjewadi
Hinjewadi
Kothrud
Kalyani Nagar
Karve Nagar
Koregaon Park
Kharadi
Katraj
Kondhwa
Khadki
Laxmi Road
Magarpatta
Mundhwa
Mangalwar Peth
MG Road
Mumbai - Bangalore Highway
Mundhwa - Kharadi Road
Municipal Colony
Nagar Road
Nagar - Pune Highway
New Airport Road
Nana Peth
Narayan Peth
Narhe
NIA Campus Road
Pirangut
Pune - Solapur Highway
Pimple Saudagar
Pune Central City
Pune
Ragvilas Society
Revenue Colony
Senapati Bapat Road
Shaniwar Peth
Somwar Peth
Sadashiv Peth
Sinhagad Road
Swargate
Sangamwadi Road
Shukrawar Peth
Sky Lounge
Saswad Road
Swargate
University Road
Viman Nagar
Vitthalrao Navale Rd
Vishrantwadi
Wadgaon Sheri
Wanowrie
Ward No. 8
Wagholi
Yerwada


In [139]:
print(pune_venues.shape)
pune_venues.head()

(627, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0        Aundh              18.561883               73.810196   
1        Aundh              18.561883               73.810196   
2        Aundh              18.561883               73.810196   
3        Aundh              18.561883               73.810196   
4        Aundh              18.561883               73.810196   

                    Venue  Venue Latitude  Venue Longitude  \
0  Picantos Mexican Grill       18.560654        73.812447   
1             Venkys Xprs       18.560550        73.808964   
2          Baker's Basket       18.560704        73.813100   
3                The Loft       18.562714        73.807630   
4        Hyderabadi Spice       18.560594        73.810108   

         Venue Category  
0    Mexican Restaurant  
1  Fast Food Restaurant  
2            Restaurant  
3     Indian Restaurant  
4     Indian Restaurant

There are 627 total restaurants in Pune.

In [140]:
# Grouping
pune_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                             
Anand Nagar                           7                       7      7   
Aundh                                23                      23     23   
Baner                                30                      30     30   
Budhwar Peth                          5                       5      5   
Bund Garden Road                     30                      30     30   
...                                 ...                     ...    ...   
Wadgaon Sheri                         2                       2      2   
Wagholi                               5                       5      5   
Wanowrie                             13                      13     13   
Ward No. 8                            6                       6      6   
Yerwada                               2                       2      2   

                  Venue Latitude  Venue Longitude  Venue Category  
Neighborhood                                                       
Anand Nagar                    7                7               7  
Aundh                         23               23              23  
Baner                         30               30              30  
Budhwar Peth                   5                5               5  
Bund Garden Road              30               30              30  
...                          ...              ...             ...  
Wadgaon Sheri                  2                2               2  
Wagholi                        5                5               5  
Wanowrie                      13               13              13  
Ward No. 8                     6                6               6  
Yerwada                        2                2               2  

[63 rows x 6 columns]

In [141]:
print('There are {} uniques categories.'.format(len(pune_venues['Venue Category'].unique())))

There are 54 uniques categories.


In [142]:
# onehot encoding
pune_onehot = pd.get_dummies(pune_venues[['Venue Category']], prefix="", prefix_sep="")

pune_onehot['Neighborhood'] = pune_venues['Neighborhood'] 

fixed_columns = [pune_onehot.columns[-1]] + list(pune_onehot.columns[:-1])
pune_onehot = pune_onehot[fixed_columns]

pune_onehot.head()

Neighborhood  Afghan Restaurant  American Restaurant  Asian Restaurant  \
0        Aundh                  0                    0                 0   
1        Aundh                  0                    0                 0   
2        Aundh                  0                    0                 0   
3        Aundh                  0                    0                 0   
4        Aundh                  0                    0                 0   

   BBQ Joint  Bakery  Bistro  Breakfast Spot  Buffet  Burger Joint  ...  \
0          0       0       0               0       0             0  ...   
1          0       0       0               0       0             0  ...   
2          0       0       0               0       0             0  ...   
3          0       0       0               0       0             0  ...   
4          0       0       0               0       0             0  ...   

   Pizza Place  Punjabi Restaurant  Restaurant  Sandwich Place  \
0            0                   0           0               0   
1            0                   0           0               0   
2            0                   0           1               0   
3            0                   0           0               0   
4            0                   0           0               0   

   Seafood Restaurant  Snack Place  South Indian Restaurant  Steakhouse  \
0                   0            0                        0           0   
1                   0            0                        0           0   
2                   0            0                        0           0   
3                   0            0                        0           0   
4                   0            0                        0           0   

   Thai Restaurant  Vegetarian / Vegan Restaurant  
0                0                              0  
1                0                              0  
2                0                              0  
3                0                              0  
4                0                              0  

[5 rows x 55 columns]

In [143]:
pune_onehot.shape

(627, 55)

In [144]:
pune_grouped = pune_onehot.groupby('Neighborhood').mean().reset_index()
pune_grouped

Neighborhood  Afghan Restaurant  American Restaurant  \
0        Anand Nagar                0.0                  0.0   
1              Aundh                0.0                  0.0   
2              Baner                0.0                  0.0   
3       Budhwar Peth                0.0                  0.0   
4   Bund Garden Road                0.0                  0.0   
..               ...                ...                  ...   
58     Wadgaon Sheri                0.0                  0.0   
59           Wagholi                0.0                  0.0   
60          Wanowrie                0.0                  0.0   
61        Ward No. 8                0.0                  0.0   
62           Yerwada                0.0                  0.0   

    Asian Restaurant  BBQ Joint    Bakery    Bistro  Breakfast Spot    Buffet  \
0                0.0   0.000000  0.285714  0.000000        0.000000  0.000000   
1                0.0   0.000000  0.043478  0.000000        0.086957  0.000000   
2                0.0   0.000000  0.000000  0.000000        0.066667  0.000000   
3                0.0   0.000000  0.200000  0.000000        0.000000  0.000000   
4                0.0   0.033333  0.100000  0.033333        0.033333  0.033333   
..               ...        ...       ...       ...             ...       ...   
58               0.0   0.000000  0.000000  0.000000        0.500000  0.000000   
59               0.0   0.000000  0.000000  0.000000        0.000000  0.000000   
60               0.0   0.000000  0.153846  0.000000        0.076923  0.000000   
61               0.0   0.000000  0.166667  0.000000        0.000000  0.000000   
62               0.0   0.000000  0.000000  0.000000        0.000000  0.000000   

    Burger Joint  ...  Pizza Place  Punjabi Restaurant  Restaurant  \
0            0.0  ...     0.142857                 0.0    0.000000   
1            0.0  ...     0.000000                 0.0    0.086957   
2            0.0  ...     0.000000                 0.0    0.066667   
3            0.0  ...     0.000000                 0.0    0.000000   
4            0.0  ...     0.066667                 0.0    0.033333   
..           ...  ...          ...                 ...         ...   
58           0.0  ...     0.000000                 0.0    0.000000   
59           0.0  ...     0.200000                 0.0    0.200000   
60           0.0  ...     0.000000                 0.0    0.000000   
61           0.0  ...     0.000000                 0.0    0.000000   
62           0.0  ...     0.000000                 0.0    0.000000   

    Sandwich Place  Seafood Restaurant  Snack Place  South Indian Restaurant  \
0         0.000000                 0.0     0.142857                 0.000000   
1         0.000000                 0.0     0.086957                 0.000000   
2         0.000000                 0.0     0.133333                 0.066667   
3         0.000000                 0.0     0.000000                 0.000000   
4         0.000000                 0.0     0.000000                 0.000000   
..             ...                 ...          ...                      ...   
58        0.000000                 0.0     0.000000                 0.000000   
59        0.000000                 0.0     0.000000                 0.000000   
60        0.076923                 0.0     0.076923                 0.000000   
61        0.000000                 0.0     0.333333                 0.000000   
62        0.000000                 0.0     0.500000                 0.000000   

    Steakhouse  Thai Restaurant  Vegetarian / Vegan Restaurant  
0          0.0              0.0                       0.000000  
1          0.0              0.0                       0.043478  
2          0.0              0.0                       0.066667  
3          0.0              0.0                       0.000000  
4          0.0              0.0                       0.000000  
..         ...              ...                            ...  
58       

In [145]:
pune_grouped.shape

(63, 55)

In [146]:
# Listing down top 5 types of restaurants in each area
num_top_venues = 5

for hood in pune_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = pune_grouped[pune_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anand Nagar----
                  venue  freq
0                Bakery  0.29
1  Fast Food Restaurant  0.29
2           Snack Place  0.14
3                  Café  0.14
4           Pizza Place  0.14


----Aundh----
                  venue  freq
0     Indian Restaurant  0.26
1  Fast Food Restaurant  0.13
2            Restaurant  0.09
3        Breakfast Spot  0.09
4           Snack Place  0.09


----Baner----
                           venue  freq
0                           Café  0.20
1           Fast Food Restaurant  0.20
2              Indian Restaurant  0.20
3                    Snack Place  0.13
4  Vegetarian / Vegan Restaurant  0.07


----Budhwar Peth----
                     venue  freq
0        Indian Restaurant   0.6
1        Indian Sweet Shop   0.2
2                   Bakery   0.2
3  North Indian Restaurant   0.0
4       Italian Restaurant   0.0


----Bund Garden Road----
                  venue  freq
0     Indian Restaurant  0.33
1  Fast Food Restaurant  0.10
2               

4         Food Court  0.17


----Somwar Peth----
                     venue  freq
0           Sandwich Place  0.14
1       Seafood Restaurant  0.14
2        Indian Restaurant  0.14
3  Comfort Food Restaurant  0.14
4                     Café  0.14


----Swargate----
                            venue  freq
0               Indian Restaurant  0.50
1               French Restaurant  0.25
2  Multicuisine Indian Restaurant  0.25
3         New American Restaurant  0.00
4               Indian Sweet Shop  0.00


----University Road----
                venue  freq
0   Indian Restaurant  0.33
1    Asian Restaurant  0.17
2  Italian Restaurant  0.17
3        Burger Joint  0.17
4  Chinese Restaurant  0.17


----Viman Nagar----
                     venue  freq
0        Indian Restaurant  0.33
1              Snack Place  0.33
2                   Bakery  0.17
3            Burrito Place  0.17
4  New American Restaurant  0.00


----Vishrantwadi----
                     venue  freq
0        Indian Restaura

In [147]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [148]:
# Create area-wise top 10 most common type of restaurants
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# New dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pune_grouped['Neighborhood']

for ind in np.arange(pune_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pune_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0       Anand Nagar                Bakery  Fast Food Restaurant   
1             Aundh     Indian Restaurant  Fast Food Restaurant   
2             Baner     Indian Restaurant                  Café   
3      Budhwar Peth     Indian Restaurant                Bakery   
4  Bund Garden Road     Indian Restaurant    Chinese Restaurant   

  3rd Most Common Venue          4th Most Common Venue 5th Most Common Venue  \
0                  Café                    Snack Place           Pizza Place   
1        Breakfast Spot                    Snack Place                  Café   
2  Fast Food Restaurant                    Snack Place            Restaurant   
3     Indian Sweet Shop  Vegetarian / Vegan Restaurant              Creperie   
4                Bakery           Fast Food Restaurant           Pizza Place   

     6th Most Common Venue          7th Most Common Venue  \
0  Comfort Food Restaurant                     Food Truck   
1               Restaurant      Middle Eastern Restaurant   
2           Breakfast Spot  Vegetarian / Vegan Restaurant   
3               Food Truck                     Food Stand   
4                BBQ Joint                         Bistro   

     8th Most Common Venue          9th Most Common Venue  \
0               Food Stand                     Food Court   
1        Korean Restaurant  Vegetarian / Vegan Restaurant   
2  South Indian Restaurant                           Food   
3               Food Court                           Food   
4               Restaurant                 Breakfast Spot   

     10th Most Common Venue  
0                      Food  
1                    Bakery  
2   Comfort Food Restaurant  
3      Fast Food Restaurant  
4  Maharashtrian Restaurant

### k-means clustering with 3 clusters

In [149]:
# Number of clusters
kclusters = 3

pune_grouped_clustering = pune_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pune_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [150]:
# Adding labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pune_merged = df_pune


pune_merged = pune_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Area')

pune_merged.head()

Area   Latitude  Longitude  Cluster Labels  \
0             Aundh  18.561883  73.810196             0.0   
3       Anand Nagar  18.478490  73.821326             1.0   
5          Balewadi  18.582027  73.768983             NaN   
6             Baner  18.564243  73.776857             1.0   
7  Bund Garden Road  18.533404  73.878625             0.0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0     Indian Restaurant  Fast Food Restaurant        Breakfast Spot   
3                Bakery  Fast Food Restaurant                  Café   
5                   NaN                   NaN                   NaN   
6     Indian Restaurant                  Café  Fast Food Restaurant   
7     Indian Restaurant    Chinese Restaurant                Bakery   

  4th Most Common Venue 5th Most Common Venue    6th Most Common Venue  \
0           Snack Place                  Café               Restaurant   
3           Snack Place           Pizza Place  Comfort Food Restaurant   
5                   NaN                   NaN                      NaN   
6           Snack Place            Restaurant           Breakfast Spot   
7  Fast Food Restaurant           Pizza Place                BBQ Joint   

           7th Most Common Venue    8th Most Common Venue  \
0      Middle Eastern Restaurant        Korean Restaurant   
3                     Food Truck               Food Stand   
5                            NaN                      NaN   
6  Vegetarian / Vegan Restaurant  South Indian Restaurant   
7                         Bistro               Restaurant   

           9th Most Common Venue    10th Most Common Venue  
0  Vegetarian / Vegan Restaurant                    Bakery  
3                     Food Court                      Food  
5                            NaN                       NaN  
6                           Food   Comfort Food Restaurant  
7                 Breakfast Spot  Maharashtrian Restaurant

In [151]:
pune_merged.dropna(axis=0, inplace=True)

In [152]:
pune_merged['Cluster Labels'] = pune_merged['Cluster Labels'].astype(int)
pune_merged.head(20)

Area   Latitude  Longitude  Cluster Labels  \
0                    Aundh  18.561883  73.810196               0   
3              Anand Nagar  18.478490  73.821326               1   
6                    Baner  18.564243  73.776857               1   
7         Bund Garden Road  18.533404  73.878625               0   
10            Budhwar Peth  18.517560  73.858037               0   
11                   Baner  18.564243  73.776857               1   
12            Central Pune  18.520306  73.854319               0   
17        Dhole Patil Road  18.534908  73.876258               0   
18         Deccan Gymkhana  18.515893  73.841160               0   
21                  Dapodi  18.580846  73.832775               1   
23  Fergusson College Road  18.524148  73.838591               0   
25            Fatima Nagar  18.504510  73.902129               0   
27                Ghorpadi  18.526854  73.900096               0   
28                Gultekdi  18.493894  73.867633               1   
29                Hadapsar  18.526967  73.927825               2   
30               Hinjewadi  18.592094  73.757142               0   
31               Hinjewadi  18.592094  73.757142               0   
32                 Kothrud  18.503889  73.807673               0   
33           Kalyani Nagar  18.548138  73.902551               1   
34             Karve Nagar  18.489404  73.821324               1   

      1st Most Common Venue          2nd Most Common Venue  \
0         Indian Restaurant           Fast Food Restaurant   
3                    Bakery           Fast Food Restaurant   
6         Indian Restaurant                           Café   
7         Indian Restaurant             Chinese Restaurant   
10        Indian Restaurant                         Bakery   
11        Indian Restaurant                           Café   
12              Snack Place              Indian Restaurant   
17        Indian Restaurant                         Bakery   
18        Indian Restaurant                           Café   
21               Food Court                     Restaurant   
23        Indian Restaurant               Asian Restaurant   
25        Indian Restaurant                 Breakfast Spot   
27  Comfort Food Restaurant              Indian Restaurant   
28           Breakfast Spot                   Burger Joint   
29              Snack Place  Vegetarian / Vegan Restaurant   
30               Restaurant              Indian Restaurant   
31               Restaurant              Indian Restaurant   
32        Indian Restaurant                           Café   
33                     Café              Indian Restaurant   
34               Food Truck                    Snack Place   

            3rd Most Common Venue          4th Most Common Venue  \
0                  Breakfast Spot                    Snack Place   
3                            Café                    Snack Place   
6            Fast Food Restaurant                    Snack Place   
7                          Bakery           Fast Food Restaurant   
10              Indian Sweet Shop  Vegetarian / Vegan Restaurant   
11           Fast Food Restaurant                    Snack Place   
12                         Bakery                  Burrito Place   
17           Fast Food Restaurant             Chinese Restaurant   
18  Vegetarian / Vegan Restaurant           Fast Food Restaurant   
21                 Breakfast Spot                      Cafeteria   
23                 Sandwich Place                     Restaurant   
25                    Pizza Place                           Café   
27  Vegetarian / Vegan Restaurant                     Food Truck   
28  Vegetarian / Vegan Restaurant                       Creperie   
29             Chinese Restaurant                     Food Truck   
30                 Breakfast Spot       Mediterranean Restaurant   
31                 Breakfast Spot       Mediterranean Restaurant   
32            American Restaurant                    Snack Place  

In [153]:
# Map Creation
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Latitude'], pune_merged['Longitude'], pune_merged['Area'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### The results from k-means clustering show that: 

- Cluster 0 (Red): Areas with high number of Indian restaurants
- Cluster 1 (Purple): Areas with high number of snacks/breakfast outlets
- Cluster 2 (Green): Areas with high concentration of food trucks/Food courts/Fast food

In [154]:
# Cluster 0
pune_merged.loc[pune_merged['Cluster Labels'] == 0, pune_merged.columns[[1] + list(range(4, pune_merged.shape[1]))]]

Latitude           1st Most Common Venue  \
0    18.561883               Indian Restaurant   
7    18.533404               Indian Restaurant   
10   18.517560               Indian Restaurant   
12   18.520306                     Snack Place   
17   18.534908               Indian Restaurant   
18   18.515893               Indian Restaurant   
23   18.524148               Indian Restaurant   
25   18.504510               Indian Restaurant   
27   18.526854         Comfort Food Restaurant   
30   18.592094                      Restaurant   
31   18.592094                      Restaurant   
32   18.503889               Indian Restaurant   
36   18.550518               Indian Restaurant   
37   18.453679               Indian Restaurant   
41   18.568175               Indian Restaurant   
42   18.508405               Indian Restaurant   
45   18.514894               Indian Restaurant   
51   18.515963               Indian Restaurant   
52   18.459712                     Snack Place   
53   18.545907  Multicuisine Indian Restaurant   
54   18.508672               Indian Restaurant   
58   18.573073               Indian Restaurant   
59   18.518357               Indian Restaurant   
60   18.515579                     Snack Place   
61   18.446015               Indian Restaurant   
68   18.598155               Indian Restaurant   
69   18.510289                     Snack Place   
72   18.520306                     Snack Place   
75   18.520306                     Snack Place   
76   18.520306                     Snack Place   
81   18.519284                     Snack Place   
83   18.510776               Indian Restaurant   
86   18.500728               Indian Restaurant   
87   18.541712        Mediterranean Restaurant   
88   18.511354               Indian Restaurant   
94   18.443630               Indian Restaurant   
96   18.500728               Indian Restaurant   
99   18.537493               Indian Restaurant   
101  18.520306                     Snack Place   
103  18.572630               Indian Restaurant   
107  18.520306                     Snack Place   
108  18.580630   Vegetarian / Vegan Restaurant   

              2nd Most Common Venue          3rd Most Common Venue  \
0              Fast Food Restaurant                 Breakfast Spot   
7                Chinese Restaurant                         Bakery   
10                           Bakery              Indian Sweet Shop   
12                Indian Restaurant                         Bakery   
17                           Bakery           Fast Food Restaurant   
18                             Café  Vegetarian / Vegan Restaurant   
23                 Asian Restaurant                 Sandwich Place   
25                   Breakfast Spot                    Pizza Place   
27                Indian Restaurant  Vegetarian / Vegan Restaurant   
30                Indian Restaurant                 Breakfast Spot   
31                Indian Restaurant                 Breakfast Spot   
32                             Café            American Restaurant   
36             Fast Food Restaurant        South Indian Restaurant   
37                             Food                 Breakfast Spot   
41                           Bakery             Punjabi Restaurant   
42                             Café                     Restaurant   
45                             Café                    Snack Place   
51             Fast Food Restaurant                 Breakfast Spot   
52                Indian Restaurant                         Bakery   
53                  Thai Restaurant              Indian Restaurant   
54    Vegetarian / Vegan Restaurant                    Snack Place   
58                 Asian Restaurant           Fast Food Restaurant   
59                      Snack Place                 Sandwich Place   
60                Indian Restaurant                         Bakery   
61                   Breakfast Spot  Vegetarian / Vegan Restaurant   
68                             Café

In [155]:
# Cluster 1
pune_merged.loc[pune_merged['Cluster Labels'] == 1, pune_merged.columns[[1] + list(range(4, pune_merged.shape[1]))]]

Latitude 1st Most Common Venue          2nd Most Common Venue  \
3    18.478490                Bakery           Fast Food Restaurant   
6    18.564243     Indian Restaurant                           Café   
11   18.564243     Indian Restaurant                           Café   
21   18.580846            Food Court                     Restaurant   
28   18.493894        Breakfast Spot                   Burger Joint   
33   18.548138                  Café              Indian Restaurant   
34   18.489404            Food Truck                    Snack Place   
35   18.537553     Indian Restaurant               Asian Restaurant   
40   18.477746            Restaurant  Vegetarian / Vegan Restaurant   
48   18.533670       Thai Restaurant                         Bakery   
56   18.557350                  Café                     Restaurant   
57   18.589521            Food Truck                    Snack Place   
62   18.573425                  Café                     Food Stand   
79   18.525574            Food Court                     Restaurant   
82   18.521766                  Café        Comfort Food Restaurant   
85   18.500368            Food Truck  Vegetarian / Vegan Restaurant   
93   18.511762            Food Truck               Asian Restaurant   
102  18.518231                  Café              Indian Restaurant   
105  18.550441        Breakfast Spot           Fast Food Restaurant   
106  18.488368  Fast Food Restaurant                           Café   

             3rd Most Common Venue          4th Most Common Venue  \
3                             Café                    Snack Place   
6             Fast Food Restaurant                    Snack Place   
11            Fast Food Restaurant                    Snack Place   
21                  Breakfast Spot                      Cafeteria   
28   Vegetarian / Vegan Restaurant                       Creperie   
33            Fast Food Restaurant  Vegetarian / Vegan Restaurant   
34                          Bakery                 Breakfast Spot   
35                          Bakery                       Creperie   
40           Hyderabadi Restaurant              Indian Restaurant   
48                  Breakfast Spot       Maharashtrian Restaurant   
56                      Food Truck             Italian Restaurant   
57                      Restaurant           Fast Food Restaurant   
62            Fast Food Restaurant                     Restaurant   
79                     Snack Place                 Sandwich Place   
82               Indian Restaurant             Seafood Restaurant   
85         Comfort Food Restaurant              French Restaurant   
93               Indian Restaurant                     Food Court   
102                         Bakery           Fast Food Restaurant   
105  Vegetarian / Vegan Restaurant        Comfort Food Restaurant   
106                         Bakery                    Snack Place   

             5th Most Common Venue          6th Most Common Venue  \
3                      Pizza Place        Comfort Food Restaurant   
6                       Restaurant                 Breakfast Spot   
11                      Restaurant                 Breakfast Spot   
21              Chinese Restaurant  Vegetarian / Vegan Restaurant   
28               French Restaurant                     Food Truck   
33                Asian Restaurant                 Breakfast Spot   
34   Vegetarian / Vegan Restaurant        Comfort Food Restaurant   
35                            Café             Italian Restaurant   
40                     Pizza Place                 Breakfast Spot   
48     Eastern European Restaurant                       Creperie   
56                     Pizza Place              Indian Restaurant   
57   Vegetarian / Vegan Restaurant             Chinese Restaurant   
62              Chinese Restaurant                     Food Truck   
79                     Pizza Place  Vegetarian / Vegan Restaurant   
82                  Sandwich 

In [156]:
# Cluster 2
pune_merged.loc[pune_merged['Cluster Labels'] == 2, pune_merged.columns[[1] + list(range(4, pune_merged.shape[1]))]]

Latitude 1st Most Common Venue          2nd Most Common Venue  \
29   18.526967           Snack Place  Vegetarian / Vegan Restaurant   
49   18.524339           Snack Place  Vegetarian / Vegan Restaurant   
63   18.511282           Snack Place                           Café   
109  18.550361           Snack Place                     Donut Shop   

             3rd Most Common Venue    4th Most Common Venue  \
29              Chinese Restaurant               Food Truck   
49              Chinese Restaurant               Food Truck   
63                  Breakfast Spot  Comfort Food Restaurant   
109  Vegetarian / Vegan Restaurant       Chinese Restaurant   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
29             Food Stand            Food Court                  Food   
49             Food Stand            Food Court                  Food   
63             Food Truck            Food Stand            Food Court   
109            Food Truck            Food Stand            Food Court   

    8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
29   Fast Food Restaurant    Falafel Restaurant  Eastern European Restaurant  
49   Fast Food Restaurant    Falafel Restaurant  Eastern European Restaurant  
63                   Food  Fast Food Restaurant           Falafel Restaurant  
109                  Food  Fast Food Restaurant           Falafel Restaurant

### Conclusion

As observations noted from the map in the Results section, most Indian restaurants are located around the central and northern parts of Pune. There is a lot of potential to open Indian Restaurants in the south and south east part of the city with little to no competition. Another opportunity is with food trucks, as they are rare in the north-west part of the city.